In [2]:
# Step 1: Import required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [4]:
# Step 2: Generate Synthetic Data

# Set dimensions
num_samples = 300  # 100 per class
image_height, image_width = 28, 28
num_classes = 3

# Create synthetic grayscale image data
X = np.random.rand(num_samples, 1, image_height, image_width).astype(np.float32)

# Labels: 0=Cat, 1=Dog, 2=Raccoon
y = np.array([0]*100 + [1]*100 + [2]*100)

# Shuffle data
indices = np.random.permutation(num_samples)
X = X[indices]
y = y[indices]


In [5]:
# Step 3: Define a simple feedforward neural network in PyTorch
import torch.nn as nn

class AnimalClassifier(nn.Module):
    def __init__(self):
        super(AnimalClassifier, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 128)   # First hidden layer
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)        # Second hidden layer
        self.relu2 = nn.ReLU()
        self.output = nn.Linear(64, 3)       # Output layer for 3 classes

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        return self.output(x)  # logits (no softmax here; we use CrossEntropyLoss)

# Instantiate the model and print summary
model = AnimalClassifier()
print(model)


AnimalClassifier(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (relu2): ReLU()
  (output): Linear(in_features=64, out_features=3, bias=True)
)


In [9]:
# Step 4: Set up the loss function and optimizer

import torch.optim as optim

# CrossEntropyLoss is perfect for multi-class classification problems.
# It combines softmax + negative log likelihood internally.
criterion = nn.CrossEntropyLoss()

# We'll use the Adam optimizer to update model weights.
# It's known for fast convergence in many deep learning tasks.
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [12]:
# Step 4.4: Simulate grayscale 28x28 images and labels

import numpy as np

num_samples = 300
image_height, image_width = 28, 28
num_classes = 3  # 0=Cat, 1=Dog, 2=Raccoon

# Generate synthetic grayscale image data
X_train = np.random.rand(num_samples, image_height, image_width, 1)

# Generate class labels: 100 samples each of 3 classes
y_train = np.array([0]*100 + [1]*100 + [2]*100)

# Shuffle the data
indices = np.random.permutation(num_samples)
X_train = X_train[indices]
y_train = y_train[indices]


In [13]:
# Step 4.5: Convert NumPy arrays to PyTorch tensors for training

import torch

# Convert input data to float32 and labels to long (int64) for classification
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)


In [14]:
# Step 5: Training loop (run for a few epochs to train the model)

epochs = 10  # Number of passes through the full dataset

for epoch in range(epochs):
    model.train()  # Put the model in training mode

    optimizer.zero_grad()        # Clear gradients from the previous step
    outputs = model(X_train_tensor)  # Forward pass
    loss = criterion(outputs, y_train_tensor)  # Calculate loss

    loss.backward()  # Backpropagation
    optimizer.step()  # Update model weights

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [1/10], Loss: 1.1054
Epoch [2/10], Loss: 1.0945
Epoch [3/10], Loss: 1.0886
Epoch [4/10], Loss: 1.0819
Epoch [5/10], Loss: 1.0739
Epoch [6/10], Loss: 1.0647
Epoch [7/10], Loss: 1.0535
Epoch [8/10], Loss: 1.0410
Epoch [9/10], Loss: 1.0267
Epoch [10/10], Loss: 1.0106


In [15]:
# Step 6: Evaluate the model on new unseen data

# Generate 5 new random grayscale images (like new test set)
X_test = np.random.rand(5, image_height, image_width, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# Set model to evaluation mode
model.eval()

# Get predictions (no gradient tracking needed during inference)
with torch.no_grad():
    predictions = model(X_test_tensor)

# Interpret predictions
class_names = ['Cat', 'Dog', 'Raccoon']
predicted_classes = torch.argmax(predictions, dim=1)
confidences = torch.nn.functional.softmax(predictions, dim=1)

# Print results
for i, (pred_class, conf) in enumerate(zip(predicted_classes, confidences)):
    label = class_names[pred_class]
    confidence = conf[pred_class].item()
    print(f"Test Image {i+1}: Predicted as '{label}' with {confidence:.2f} confidence")


Test Image 1: Predicted as 'Dog' with 0.36 confidence
Test Image 2: Predicted as 'Raccoon' with 0.37 confidence
Test Image 3: Predicted as 'Cat' with 0.35 confidence
Test Image 4: Predicted as 'Raccoon' with 0.34 confidence
Test Image 5: Predicted as 'Dog' with 0.36 confidence
